In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import urllib.parse

In [2]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [3]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [4]:
%%capture pwd
!pwd

In [5]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
uid=urllib.parse.quote(uid)
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('what-are-the-big-policy-challenges-facing-scotlands-new-first-minister',
 300,
 500)

# Fig 1

In [6]:
df = pd.read_excel("raw/GDP+1st+Estimate+2022+Q4+-+Tables.xlsx",sheet_name='Sheet1',usecols='A:H').ffill()
df['year']=df['Unnamed: 0'].astype(int).astype(str) + '-'+df['Unnamed: 1'].str.replace(' ','').replace('Q1','03').replace('Q2','06').replace('Q3','09').replace('Q4','12')+'-01'
df=df.drop(['Unnamed: 0','Unnamed: 1'],axis=1).interpolate()
df.columns=['sc','uk','sc2','uk2','sc3','uk3','year']

df2 = pd.read_excel("raw/GDP+1st+Estimate+2022+Q4+-+Tables.xlsx",sheet_name='Sheet1',usecols='Q:V').ffill()
df2['year']=df2['Unnamed: 16'].astype(int).astype(str) + '-'+df2['Unnamed: 17'].str.replace(' ','').replace('Q1','03').replace('Q2','06').replace('Q3','09').replace('Q4','12')+'-01'
df2=df2.drop(['Unnamed: 16','Unnamed: 17'],axis=1).interpolate()
df2.columns=['sc4','uk4','sc5','uk5','year']

df=df.join(df2.set_index('year'),on='year')

In [7]:
f = "fig1_gdp"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,sc,uk,sc2,uk2,sc3,uk3,year,sc4,uk4,sc5,uk5
0,78.177229,77.0,85.484438,82.397004,85.484438,82.397004,1998-03-01,72.687035,67.3,83.911142,78.438228
1,78.509482,77.3,85.847747,82.718031,85.847747,82.718031,1998-06-01,72.973425,67.7,84.241755,78.904429
2,78.426852,77.5,85.757393,82.932049,85.757393,82.932049,1998-09-01,72.878243,67.9,84.131875,79.137529
3,78.682215,77.9,86.036625,83.360086,86.036625,83.360086,1998-12-01,73.097101,68.3,84.384529,79.603730
4,78.730274,78.4,86.089176,83.895131,86.089176,83.895131,1999-03-01,73.123300,68.8,84.414773,80.186480


In [10]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = base.mark_line(color=colors["eco-light-blue"]).encode(
    y=alt.Y(
        "sc2:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Index (2007 = 100)",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        # scale=alt.Scale(domain=[70, 140]),
    )
)
line2 = line.mark_line(color=colors["eco-red"]).encode(y=alt.Y(
    "uk2:Q",
    sort=[],
))
line3 = line.mark_line(color=colors["eco-red"]).encode(
    y=alt.Y("sc4:Q", sort=[]))
line4 = line.mark_line(color=colors["eco-red"]).encode(
    y=alt.Y("uk4:Q", sort=[]))
axis1 = (
    alt.Chart(pd.DataFrame([{"x": "1998-03-01", "y": 100}, {"x": "2022-12-01", "y": 100}]))
    .mark_line(strokeWidth=1, opacity=0.4, color=colors["eco-gray"])
    .encode(x=alt.X("x:T", sort=[]), y=alt.Y("y:Q",sort=[],scale=alt.Scale(domain=[70, 140])))
)
axis2 = (
    alt.Chart(pd.DataFrame([{"x": "2007-01-01", "y": 70}, {"x": "2007-01-01", "y": 140}]))
    .mark_line(strokeWidth=1, opacity=0.4, color=colors["eco-gray"])
    .encode(x=alt.X("x:T", sort=[]), y=alt.Y("y:Q",sort=[],scale=alt.Scale(domain=[70, 140])))
)
# fit = base.transform_filter('year(datum.year)<2007').transform_regression("year", "uk2")\
fit1=alt.Chart(pd.DataFrame([{'x':'1998-01-01', 'y': 84.03},{'x':'2022-12-01', 'y':127.47}]))\
.mark_line(color=colors["eco-light-blue"],strokeDash=[5,5]).encode(x='x:T',y='y:Q')
fit2=alt.Chart(pd.DataFrame([{'x':'1998-01-01', 'y':82.67952840300109},{'x':'2022-12-01', 'y':129.72132904608787}]))\
.mark_line(color=colors["eco-red"],strokeDash=[5,5]).encode(x='x:T',y='y:Q')
layer1 = (((axis1+axis2+fit1+fit2+line + line2).properties(
    height=300, width=400)).configure_view(stroke=None).properties(title=""))
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

Fontconfig error: Cannot load default config file
Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 2

In [11]:
df = pd.read_excel("raw/Scotland_s-Economic-and-Fiscal-Forecasts-December-2022-Summary-Figures-Revised-February-2023.xlsx",
                   sheet_name='Figure 6', skiprows=22, nrows=2)
df=df.T.iloc[1:]
df.index=range(2017,2028)
df.index.name='year'
df.columns=['dec','pol']
df=df.reset_index()

In [12]:
f = "fig2_tax"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,dec,pol
0,2017,-97.0,93.98912
1,2018,119.0,385.113245
2,2019,147.963209,621.157822
3,2020,96.0,641.983219
4,2021,-256.772002,757.383652


In [13]:
base = alt.Chart(f2).encode(
    x=alt.X(
        "year:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            title="tax year 2027/28",
            titleY=-15,
            # titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
            format='.0f'
        ),
    )
)
line = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "dec:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="£ million",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            format='.0f'
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
axis = (
    alt.Chart(pd.DataFrame([{"x": 2017, 'y':0}, {"x": 2027, "y": 0}]))
    .mark_line(strokeWidth=1, opacity=0.4, color=colors["eco-gray"])
    .encode(x=alt.X("x:Q", sort=[]), y=alt.Y("y:Q",sort=[]))
)
labels = (
    alt.Chart(pd.DataFrame([{"x": 2020, 'y':720,'t':'Policy only'}, {"x": 2023, "y": 400,'t':'December 2022'}]))
    .mark_text(align='right')
    .encode(x=alt.X("x:Q", sort=[]), y=alt.Y("y:Q",sort=[]),text='t:N',color=alt.Color('t:N',legend=None,scale=alt.Scale(range=[colors["eco-turquiose"],colors["eco-dot"]])))
)
line2=base.mark_line(color=colors["eco-dot"]).encode(
    y=alt.Y(
        "pol:Q",
        sort=[]))
layer1 = (
    ((axis+line+line2+labels
     ).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file
Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 3

In [14]:
df = pd.read_excel("raw/Scotland_s-Economic-and-Fiscal-Forecasts-December-2022-Chapter-3-Economy-Figures.xlsx",
                   sheet_name='Figure 3.16', skiprows=19).dropna()
df['Period']=range(1998,2028)
df.columns=['year','earnings','p14']

In [15]:
f = "fig3_earnings"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,earnings,p14
0,1998,97.383010,95.885798
1,1999,97.192951,95.885798
2,2000,95.232190,95.885798
3,2001,96.045187,95.885798
4,2002,96.452236,95.885798


In [16]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "year:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            title="tax year 2027/28",
            titleY=-15,
            # titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
            format='.0f'
        ),
    )
)
line = base.mark_line(color=colors["eco-light-blue"]).encode(
    y=alt.Y(
        "earnings:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="£ million",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            format='.0f'
        ),
        scale=alt.Scale(domain=[90, 100]),
    )
)
axis = (
    alt.Chart(pd.DataFrame([{"x": 2017, 'y':0}, {"x": 2027, "y": 0}]))
    .mark_line(strokeWidth=1, opacity=0.4, color=colors["eco-gray"])
    .encode(x=alt.X("x:Q", sort=[]), y=alt.Y("y:Q",sort=[]))
)
area = (
    alt.Chart(pd.DataFrame([{"x": 2022, 'y':90,'y2':100}, {"x": 2028,'y':90, "y2": 100}]))
    .mark_area(opacity=0.1, color=colors["eco-light-blue"])
    .encode(x=alt.X("x:Q", sort=[]),y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="%",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            format='.0f'
        ),
        scale=alt.Scale(domain=[90, 100]),
    ),y2='y2:Q')
)
labels = (
    alt.Chart(pd.DataFrame([{"x": 2018, 'y':98.5,'t':'Scottish nominal average earnings as a share of UK'}, 
                            {"x": 2027, "y": 96.2,'t':'Pre-2014 average of share'},
                           {"x": 2026.6, "y": 99.6,'t':'Forecast'}]))
    .mark_text(align='right')
    .encode(x=alt.X("x:Q", sort=[]), y=alt.Y("y:Q",sort=[]),text='t:N',color=alt.Color('t:N',legend=None,
scale=alt.Scale(range=[colors["eco-light-blue"],colors["eco-gray"],colors["eco-light-blue"]])))
)
line2=base.mark_line(color=colors["eco-gray"]).encode(
    y=alt.Y(
        "p14:Q",
        sort=[]))
layer1 = (
    ((area+line+line2+labels
     ).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file
Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 4

In [17]:
df1=pd.DataFrame([[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032],
 [66.3199222336399,62.0069608910763,64.669322582806,57.5160575556757,57.7161242970736,55.6752824392664,52.0619465894849,51.4959201352543,46.8525974479282,45.8949616550962,47.0722869625601,46.0200604103272,40.565149378329]]).T

In [18]:
df2=pd.DataFrame([[2020,2021],[40.565149378329,42.1877553534622]]).T

In [19]:
df3=pd.DataFrame([[2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032],
 [34.9092983655333,33.3338193419107,31.758340318288,30.1828612946654,28.6073822710427,27.0319032474201,25.4564242237975,23.8809452001748,22.3054661765522,20.7299871529295,19.4861879237537,18.242388694578]]).T

In [20]:
df=pd.concat([df1,df2,df3],axis=1)
df.columns=['x1','y1','x2','y2','x3','y3']

In [21]:
f = "fig4_emissions"
f4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,x1,y1,x2,y2,x3,y3
0,2008.0,66.319922,2020.0,40.565149,2021.0,34.909298
1,2009.0,62.006961,2021.0,42.187755,2022.0,33.333819
2,2010.0,64.669323,NaN,NaN,2023.0,31.758340
3,2011.0,57.516058,NaN,NaN,2024.0,30.182861
4,2012.0,57.716124,NaN,NaN,2025.0,28.607382


In [22]:
base = alt.Chart(f4).encode(x=alt.X(
    "x1:Q",
    sort=[],
))
line = base.mark_line(color=colors["eco-light-blue"]).encode(y=alt.Y(
    "y1:Q",
    sort=[],
    scale=alt.Scale(domain=[0, 70]),
))
line2 = base.mark_line(color=colors['eco-light-blue']).encode(
    x='x2:Q', y=alt.Y('y2:Q', scale=alt.Scale(domain=[0, 70])))
line3 = base.mark_line(color=colors['eco-light-blue']).encode(
    x='x3:Q', y=alt.Y('y3:Q', scale=alt.Scale(domain=[0, 70])))
area1 = alt.Chart(
    pd.DataFrame([{
        'x': 2020,
        'y': 0,
        'y2': 70
    }, {
        'x': 2021,
        'y': 0,
        'y2': 70
    }])).mark_area(color=colors['eco-light-blue'],
                   opacity=0.1).encode(x=alt.X(
                       "x:Q",
                       sort=[],
                       axis=alt.Axis(grid=False,
                                     titleAlign="right",
                                     titleAnchor="end",
                                     title="",
                                     titleY=-15,
                                     labelColor=colors["eco-gray"],
                                     titleColor=colors["eco-gray"],
                                     tickColor=colors["eco-gray"],
                                     domainColor=colors["eco-gray"],
                                     tickCount=10,
                                     orient="bottom",
                                     labelAngle=0,
                                     format='.0f'),
                   ),
                                       y=alt.Y(
                                           "y:Q",
                                           sort=[],
                                           axis=alt.Axis(
                                               grid=False,
                                               title="Emissions (Mt CO2e)",
                                               titleX=-5,
                                               titleY=-5,
                                               titleBaseline="bottom",
                                               titleAngle=0,
                                               titleAlign="left",
                                               labelColor=colors["eco-gray"],
                                               titleColor=colors["eco-gray"],
                                               tickColor=colors["eco-gray"],
                                               domainColor=colors["eco-gray"],
                                           ),
                                           scale=alt.Scale(domain=[0, 70]),
                                       ),
                                       y2='y2:Q')
area2 = alt.Chart(
    pd.DataFrame([{
        'x': 2021,
        'y': 0,
        'y2': 70
    }, {
        'x': 2032,
        'y': 0,
        'y2': 70
    }])).mark_area(color=colors['eco-light-blue'],
                   opacity=0.05).encode(x='x:Q', y='y:Q', y2='y2:Q')
labels = alt.Chart(
    pd.DataFrame([{
        "x": 2012,
        'y': 60,
        't': '1990-2020 inventory'
    }, {
        "x": 2024,
        "y": 35,
        't': 'Current targets'
    }, {
        "x": 2020.5,
        "y": 50,
        't': 'Projected'
    }])).mark_text(align='left',color=colors['eco-light-blue']).encode(x=alt.X("x:Q", sort=[]),y=alt.Y("y:Q", sort=[]),text='t:N')
labels1 = labels.transform_filter('datum.t=="Projected"').mark_text(
    angle=270, color=colors['eco-light-blue'])
labels2 = labels.transform_filter('datum.t!="Projected"')
layer1 = (((area1 + area2 + line + line2 + line3 + labels1 +
            labels2).properties(
                height=300,
                width=400)).configure_view(stroke=None).properties(title=""))
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file
Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 5

In [23]:
df = pd.read_excel(
    "raw/regionalgrossvalueaddedbalancedbyindustryandallitlregions_(2).xlsx",
    sheet_name='Sheet1',usecols='B:X',
    skiprows=6).dropna()
df = df.T.reset_index()
df.columns=['year','abd','sco']

In [24]:
f = "fig5_aberdeen"
f5 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH",
        f5.replace("/data/", "/visualisation/").replace(".csv", ".json")))
if LOCAL:
    f5 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,abd,sco
0,1998,100.000000,100.000000
1,1999,99.548193,101.282051
2,2000,100.451807,103.846154
3,2001,105.722892,106.695157
4,2002,108.132530,108.689459


In [25]:
base = alt.Chart(f5).encode(x=alt.X(
    "year:Q",
    sort=[],
    axis=alt.Axis(
        grid=False,
        titleAlign="right",
        titleAnchor="end",
        title="",
        titleY=-15,
        # titleX=207,
        labelColor=colors["eco-gray"],
        titleColor=colors["eco-gray"],
        tickColor=colors["eco-gray"],
        domainColor=colors["eco-gray"],
        tickCount=10,
        orient="bottom",
        labelAngle=0,
        format='.0f'),
))
line = base.mark_line(color=colors["eco-light-blue"]).encode(y=alt.Y(
    "sco:Q",
    sort=[],
    axis=alt.Axis(grid=False,
                  title="Index (1998 = 100)",
                  titleX=-5,
                  titleY=-5,
                  titleBaseline="bottom",
                  titleAngle=0,
                  titleAlign="left",
                  labelColor=colors["eco-gray"],
                  titleColor=colors["eco-gray"],
                  tickColor=colors["eco-gray"],
                  domainColor=colors["eco-gray"],
                  format='.0f'),
))
axis = (alt.Chart(pd.DataFrame([{
    "x":1998,
    'y': 100
}, {
    "x": 2020,
    "y": 100
}])).mark_line(strokeWidth=1, opacity=0.4,
               color=colors["eco-gray"]).encode(x=alt.X("x:Q", sort=[]),
                                                y=alt.Y("y:Q", sort=[],
    scale=alt.Scale(domain=[90, 180]))))
labels = (alt.Chart(
    pd.DataFrame([{
        "x": 2010,
        'y': 152,
        't': 'Aberdeen Ciy and Aberdeenshire'
    }, {
        "x": 2014,
        "y": 123,
        't': 'Scotland'
    }])).mark_text(align='right').encode(
        x=alt.X("x:Q", sort=[]),
        y=alt.Y("y:Q", sort=[],),
        text='t:N',
        color=alt.Color(
            't:N',
            legend=None,
            scale=alt.Scale(
                range=[colors["eco-dot"], colors["eco-light-blue"]]))))
line2 = base.mark_line(color=colors["eco-dot"]).encode(
    y=alt.Y("abd:Q", sort=[]))
layer1 = (((axis + line + line2+labels).properties(
    height=300, width=400)).configure_view(stroke=None).properties(title=""))
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file
Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 6

In [ ]:
df = pd.read_excel(
    "raw/ukppvopendata2020.xls",
    sheet_name='Population',
    skiprows=226)
df = df.T.reset_index()
df.columns=['year','m65-','w65-','m65+','w65+','all']
df=df.iloc[2:]

In [ ]:
f = "fig6_population"
f6 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH",
        f6.replace("/data/", "/visualisation/").replace(".csv", ".json")))
if LOCAL:
    f6 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,m65-,w65-,m65+,w65+,all
2,2020,100,100,100,100,100
3,2021,100.062005,100.022012,101.507589,101.314412,100.295834
4,2022,100.689722,100.527292,103.273279,102.808521,101.058721
5,2023,100.92192,100.663739,105.270401,104.518368,101.552209
6,2024,101.11024,100.751366,107.28846,106.298178,102.016865


In [ ]:
base = alt.Chart(f6).encode(x=alt.X(
    "year:Q",
    sort=[],
    axis=alt.Axis(
        grid=False,
        titleAlign="right",
        titleAnchor="end",
        title="",
        titleY=-15,
        # titleX=207,
        labelColor=colors["eco-gray"],
        titleColor=colors["eco-gray"],
        tickColor=colors["eco-gray"],
        domainColor=colors["eco-gray"],
        tickCount=10,
        orient="bottom",
        labelAngle=0,
        format='.0f'),
))
line = base.mark_line(color=colors["eco-gray"]).encode(y=alt.Y(
    "all:Q",
    sort=[],
    axis=alt.Axis(grid=False,
                  title="Index (2010 = 100)",
                  titleX=-5,
                  titleY=-5,
                  titleBaseline="bottom",
                  titleAngle=0,
                  titleAlign="left",
                  labelColor=colors["eco-gray"],
                  titleColor=colors["eco-gray"],
                  tickColor=colors["eco-gray"],
                  domainColor=colors["eco-gray"],
                  format='.0f'),
))
axis = (alt.Chart(pd.DataFrame([{
    "x": 2020,
    'y': 100
}, {
    "x": 2120,
    "y": 100
}])).mark_line(strokeWidth=1, opacity=0.4, color=colors["eco-gray"]).encode(
    x=alt.X("x:Q", sort=[]),
    y=alt.Y("y:Q", sort=[], scale=alt.Scale(domain=[80, 220]))))
labels = (alt.Chart(
    pd.DataFrame([{
        "x": 2120,
        'y': 207,
        't': 'Men 65+'
    }, {
        "x": 2120,
        "y": 180,
        't': 'Women 65+'
    }, {
        "x": 2120,
        "y": 114,
        't': 'All'
    }, {
        "x": 2120,
        "y": 98,
        't': 'Men <65'
    }, {
        "x": 2120,
        "y": 94,
        't': 'Women <65'
    }])).mark_text(align='left', dx=5).encode(
        x=alt.X("x:Q", sort=[]),
        y=alt.Y(
            "y:Q",
            sort=[],
        ),
        text='t:N',
        color=alt.Color('t:N',
                        legend=None,
                        scale=alt.Scale(range=[
                            colors["eco-gray"], colors["eco-light-blue"],
                            colors["eco-light-blue"], colors["eco-dot"],
                            colors["eco-dot"]
                        ]))))
line2 = base.mark_line(color=colors["eco-dot"]).encode(
    y=alt.Y("w65+:Q", sort=[]))
line3 = base.mark_line(color=colors["eco-light-blue"]).encode(
    y=alt.Y("m65+:Q", sort=[]))
line4 = base.mark_line(color=colors["eco-dot"]).encode(
    y=alt.Y("w65-:Q", sort=[]))
line5 = base.mark_line(color=colors["eco-light-blue"]).encode(
    y=alt.Y("m65-:Q", sort=[]))
layer1 = (((axis + line + line2 + line3 + line4 + line5+labels).properties(
    height=300, width=400)).configure_view(stroke=None).properties(title=""))
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


# Fig 7

In [ ]:
df = pd.read_excel(
    "raw/rtisamar2023.xlsx",
    sheet_name='18. Aggregate pay (NUTS3)',
    skiprows=114,usecols='A,EQ:FM'
    )
df=df.set_index('Unnamed: 0').stack().reset_index()
df.columns=['Month','Region','Index']
df['Date'] = '28 ' + df['Month']
df['Index']=df['Index'].round(1)

In [ ]:
f = "fig7_pay"
f7 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH",
        f7.replace("/data/", "/visualisation/").replace(".csv", ".json")))
if LOCAL:
    f7 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

In [ ]:
base = alt.Chart(f7).encode(x=alt.X(
    "Date:T",
    sort=[],
    axis=alt.Axis(
        grid=False,
        titleAlign="right",
        titleAnchor="end",
        title="",
        titleY=-15,
        # titleX=207,
        labelColor=colors["eco-gray"],
        titleColor=colors["eco-gray"],
        tickColor=colors["eco-gray"],
        domainColor=colors["eco-gray"],
        tickCount=10,
        orient="bottom",
        labelAngle=0),
))
lines = base.mark_line(color=colors["eco-gray"],opacity=0.01,strokeWidth=10).encode(
    y=alt.Y(
        "Index:Q",
        sort=[],
        axis=alt.Axis(grid=False,
                      title="Aggregate pay, Index (2014 July = 100)",
                      titleX=-5,
                      titleY=-5,
                      titleBaseline="bottom",
                      titleAngle=0,
                      titleAlign="left",
                      labelColor=colors["eco-gray"],
                      titleColor=colors["eco-gray"],
                      tickColor=colors["eco-gray"],
                      domainColor=colors["eco-gray"],
                      format='.0f'),
    ),
    color=alt.Color('Region:N'),
    tooltip=['Month:N','Region:N','Index:Q']
    )
axis = (alt.Chart(
    pd.DataFrame([{
        "x": '28 July 2014',
        'y': 100
    }, {
        "x": '28 January 2023',
        "y": 100
    }])).mark_line(strokeWidth=1, opacity=0.4,
                   color=colors["eco-gray"]).encode(
                       x=alt.X("x:T", sort=[]),
                       y=alt.Y("y:Q",
                               sort=[],
                               scale=alt.Scale(domain=[80, 160]))))
tt=lines.mark_line(strokeWidth=1,opacity=0.2)
lines2 = tt.mark_line(
    strokeWidth=2, opacity=1, color=colors['eco-turquiose']).transform_filter(
        alt.FieldOneOfPredicate(
            oneOf=[
                'Shetland Islands', 'Aberdeen City and Aberdeenshire',
                'Inverness and Nairn and Moray, Badenoch and Strathspey',
                'Glasgow City', 'City of Edinburgh',
                'Perth and Kinross and Stirling'
            ],
            field='Region')).encode(color=alt.Color('Region:N', legend=None))
labels=lines2.mark_text(align='left',xOffset=5).encode(text='r:N').transform_filter('year(datum.Date)>2022').\
transform_calculate(r='datum.Region=="Inverness and Nairn and Moray, Badenoch and Strathspey"?"Inverness area":datum.Region=="City of Edinburgh"?"Edinburgh area":datum.Region=="Glasgow City"?"Glasgow area":datum.Region=="Aberdeen City and Aberdeenshire"?"Aberdeen area":datum.Region')
layer1 = (((axis + lines + tt + lines2 +
            labels).properties(
                height=300,
                width=400)).configure_view(stroke=None).properties(title=""))
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1